## Testing out Bokeh maps
### Author: Ryan Gan
### Date Created: 2018-07-13

Figuring out how to make a leaflet-style map with Bokeh.

In [2]:
# bokeh
from bokeh.plotting import figure, show, output_file
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.models import HoverTool
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
# pandas
import pandas as pd
# change lat lon projection
from pyproj import Proj, transform
# geopandas
import geopandas as gp
import pysal as ps
# math
import math

### Current Fires

Need to do some conversions of the points from lat/lon to mercator projection.

In [3]:
# load fire data
current_fires = pd.read_csv('./data/fire_locations.csv')

In [4]:
# transform lat/lon coordinates to mercator projection
fire_coords = []
for i in zip(current_fires.longitude, current_fires.latitude):
    coords = transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), i[0], i[1])
    fire_coords.append(coords)

In [5]:
# output vectors of lons and lats
lons = []
lats = []
for lon, lat in fire_coords:
    lons.append(lon)
    lats.append(lat)
    
# output vector of fire size
size_r = []
for s in current_fires.area:
    size_r.append(math.sqrt(s/50))
size = []
for s in current_fires.area:
    size.append(s)

### Shapes

Reading in shapefile and projections.

In [6]:
# read in shapefile
grid = gp.read_file('./data/grid_poly/grid_poly.shp')

In [7]:
# convert projection to mercator
grid = grid.to_crs(crs={'init':'epsg:3857'})

In [8]:
grid.head()

,fire_risk,fire_size,geometry
0,0.212892,1.0,POLYGON ((-13586638.18987055 6274861.394006577...
1,0.218032,1.0,POLYGON ((-13477205.47010762 6274861.394006577...
2,0.224476,1.0,"POLYGON ((-13258340.0305819 6274861.394006577,..."
3,0.226336,1.0,POLYGON ((-13148907.31081909 6274861.394006577...
4,0.217261,1.0,POLYGON ((-12930041.87129324 6274861.394006577...


Custom function to get the polygon coordinates.

In [9]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [10]:
# Get the Polygon x and y coordinates
grid['x'] = grid.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
grid['y'] = grid.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

In [11]:
# making copy without geometry 
grid_nogeo = grid.drop('geometry', axis=1).copy()
# round fire risk to 3 dig
grid_nogeo['fire_risk'] = grid_nogeo['fire_risk'].round(3)

In [12]:
grid_nogeo.head()

,fire_risk,fire_size,x,y
0,0.213,1.0,"[-13586638.189870546, -13477205.470107624, -13...","[6274861.394006577, 6274861.394006577, 6113255..."
1,0.218,1.0,"[-13477205.470107624, -13367772.750344815, -13...","[6274861.394006577, 6274861.394006577, 6113255..."
2,0.224,1.0,"[-13258340.030581895, -13148907.310819086, -13...","[6274861.394006577, 6274861.394006577, 6113255..."
3,0.226,1.0,"[-13148907.310819086, -13039474.591056166, -13...","[6274861.394006577, 6274861.394006577, 6113255..."
4,0.217,1.0,"[-12930041.871293245, -12820609.151530435, -12...","[6274861.394006577, 6274861.394006577, 6113255..."


In [13]:
# grid to column data source
gsource = ColumnDataSource(grid_nogeo)

In [14]:
# Let's first do some coloring magic that converts the color palet into map numbers (it's okey not to understand)
from bokeh.palettes import RdYlGn6 as palette
from bokeh.models import LogColorMapper

# Create the color mapper
color_mapper = LogColorMapper(palette=palette)

In [15]:
output_notebook()

Loading BokehJS ...

In [17]:
# initial figure
p = figure(x_range=(-14230740, -7866287), y_range=(3008561, 6207909), 
           plot_width=800, plot_height=600,
           x_axis_type="mercator", y_axis_type="mercator", 
           tools = 'pan, zoom_in, zoom_out, reset, save, wheel_zoom')

# patches
r1 = p.patches('x', 'y', source = gsource,
         fill_color = {'field': 'fire_risk', 'transform': color_mapper},
         fill_alpha = 0.3, line_color = None, line_width = 0.05)
# add fire information
p.add_tools(HoverTool(renderers=[r1], tooltips=[("Fire Risk", "@fire_risk{1.11}")]))
# add fire circles based on location
p.scatter(x = lons, y = lats, size = size_r, marker='triangle', line_color = 'red',
          fill_color = None, alpha = 0.3)


# add stamen basemap
p.add_tile(CARTODBPOSITRON)

show(p)